# Custom convnets for the DREAM ENCODE challenge

This document will outline the math for the non-standard components. 

## Ordinal likelihood

The nnet has a single univariate output $g \in R$ since I do not use a multitask output. The ordinal likelihood is defined for the Unbound, Ambigous and Bound classes as
\begin{align}
P(y=U|g)=\sigma(-g)\\
P(y=A|g)=\sigma(1-g)-\sigma(-g)\\
P(y=B|g)=1-\sigma(1-g)
\end{align}
where $\sigma$ is the logistic function. 
